In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 12 20:14:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
%pip install datasets
%pip install rouge
%pip install -U transformers

In [1]:
import torch
import logging
import transformers
import sys
import os
import nltk
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
def calculate_rouge(reference, hypothesis):
    """
    Calculate ROUGE scores for a given reference and hypothesis.

    :param reference: The reference (ground truth) summary
    :param hypothesis: The generated summary to evaluate
    :return: A dictionary containing ROUGE-1, ROUGE-2, and ROUGE-L scores
    """
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return {
        'rouge-1': scores[0]['rouge-1']['f'],
        'rouge-2': scores[0]['rouge-2']['f'],
        'rouge-l': scores[0]['rouge-l']['f']
    }

def calculate_bleu(reference, hypothesis):
    """
    Calculate the BLEU score for a given reference and hypothesis.

    :param reference (str): The reference (ground truth) summary
    :param hypothesis (str): The generated summary to evaluate
    :return (float): The BLEU score
    """

    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)
    sf = SmoothingFunction()
    return sentence_bleu(reference_tokens, hypothesis_tokens, smoothing_function=sf.method1)

def evaluate_summary(reference, hypothesis):
    """
    Evaluate a generated summary using both ROUGE and BLEU metrics.

    :param reference (str): The reference (ground truth) summary
    :param hypothesis (str): The generated summary to evaluate
    :return (float): A dictionary containing ROUGE and BLEU scores
    """
    rouge_scores = calculate_rouge(reference, hypothesis)
    bleu_score = calculate_bleu(reference, hypothesis)

    return {
        'rouge': rouge_scores,
        'bleu': bleu_score
    }


In [5]:
"""
Load model and tokenizer.
"""
# set seed before initializing model
set_seed(777)

# output_path = '/content/drive/MyDrive/Projects/CSE842_DistilBERT/checkpoints'
output_path = "G:/My Drive/Projects/CSE842_DistilBERT/checkpoints"

# model_id = "sshleifer/distilbart-xsum-12-3"
model_id = os.path.join(output_path, "checkpoint-30084")

# load model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    early_stopping=False,
).bfloat16()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '<pad>'})

0

In [6]:
"""
Load & clean WikiHow dataset.

https://huggingface.co/datasets/gursi26/wikihow-cleaned
https://github.com/mahnazkoupaee/WikiHow-Dataset
"""
def clean_dataset(dataset):
    df = pd.DataFrame(dataset)
    print(len(df))
    df = df.dropna()
    # df = df.iloc[:100]
    print(len(df))
    return Dataset.from_pandas(df)

def download_dataset():
    data_path = "G:/My Drive/Projects/CSE842_DistilBERT/wikihow-cleaned.csv"
    df = pd.read_csv(data_path)
    # dataset = load_dataset("gursi26/wikihow-cleaned")["train"]
    dataset = Dataset.from_pandas(df)
    return dataset

# load dataset
dataset = download_dataset()
dataset = clean_dataset(dataset)

# split dataset
a = dataset.train_test_split(test_size=0.25)
b = a['test'].train_test_split(test_size=0.5)
dataset = DatasetDict({
    'train': a['train'],
    'test': b['test'],
    'valid': b['train']
})
print(dataset)

214293
213892
DatasetDict({
    train: Dataset({
        features: ['summary', 'title', 'text', '__index_level_0__'],
        num_rows: 160419
    })
    test: Dataset({
        features: ['summary', 'title', 'text', '__index_level_0__'],
        num_rows: 26737
    })
    valid: Dataset({
        features: ['summary', 'title', 'text', '__index_level_0__'],
        num_rows: 26736
    })
})


In [7]:
"""
Data tokenization & collation.
"""
# data collator
label_pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8,
)

max_input_length = 256
max_target_length = 128

if model_id in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples['text']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # setup the tokenizer for targets
    labels = tokenizer(text_target=examples['summary'], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# apply tokenization
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/160419 [00:00<?, ? examples/s]

Map:   0%|          | 0/26737 [00:00<?, ? examples/s]

Map:   0%|          | 0/26736 [00:00<?, ? examples/s]

In [8]:
"""
Evaluation metrics.
"""
def compute_metrics(eval_preds):
    # prepare prediction data
    labels, preds = eval_preds.label_ids, eval_preds.predictions
    labels[labels == -100] = tokenizer.pad_token_id

    # decode
    preds_decoded = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels_decoded = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # calculate rouge scores
    rouge_scores = calculate_rouge(labels_decoded, preds_decoded)

    # return metrics
    return rouge_scores

In [9]:
"""
Create Trainer.
"""
batch_size = 32
training_args = Seq2SeqTrainingArguments(
    learning_rate=0.005,
    weight_decay=0.01,
    log_level='info',
    output_dir=output_path,
    save_strategy='epoch',
    # save_steps=500,
    save_total_limit=2,
    use_cpu=False,
    eval_strategy='epoch',
    num_train_epochs=10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # fp16=True,
    bf16=True,
)

trainer = Seq2SeqTrainer(
    model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\1849100806.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Using cpu_amp half precision backend


In [ ]:
"""
Train model.
"""
os.environ["WANDB_DISABLED"] = "true"
training_results = trainer.train()

In [ ]:
"""
Evaluate model.
"""
testing_results = trainer.evaluate(tokenized_datasets['test'])

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, title, text, __index_level_0__. If summary, title, text, __index_level_0__ are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 26737
  Batch size = 32
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
C:\Users\Admin\miniconda3\Lib\site-packages\transformers\trainer.py:3536: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_ena

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [ ]:
"""
Save trained model.
"""
trainer.save_model(output_path)